## Importing Required Libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

## User Configuration and Hyperparameters

In [ ]:
dataset_choice = input("Enter dataset ('mnist' or 'fashion'): ").strip().lower()
epochs = 50
batch_size = 64
noise_dim = 100
lr = 0.0002
save_interval = 5

half_batch = batch_size // 2

os.makedirs(f"generated_samples_{dataset_choice}", exist_ok=True)
os.makedirs(f"final_generated_images_{dataset_choice}", exist_ok=True)

## Dataset Loading and Preprocessing

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

if dataset_choice == "mnist":
    train_set = torchvision.datasets.MNIST(
        root="./data", train=True, download=True, transform=transform
    )
else:
    train_set = torchvision.datasets.FashionMNIST(
        root="./data", train=True, download=True, transform=transform
    )

dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

## Defining Generator and Discriminator Networks

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z).view(-1, 1, 28, 28)


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img.view(img.size(0), -1))

## Model Initialization and Optimizers

In [7]:
netG = Generator().to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))

## Generator Weight and Gradient Norm Monitoring

In [8]:
def print_generator_weights_and_gradients(generator):
    print("\nGenerator Layer-wise Norms")
    print("-" * 75)

    for name, param in generator.named_parameters():
        if param.grad is not None:
            weight_norm = param.data.norm(2).item()
            grad_norm = param.grad.data.norm(2).item()
            print(
                f"{name:<15} | "
                f"Weight norm: {weight_norm:.6f} | "
                f"Grad norm: {grad_norm:.6f}"
            )

## GAN Training Loop

In [ ]:
print("\nStarting Training...\n")

for epoch in range(1, epochs + 1):

    for real_imgs, _ in dataloader:
        real_imgs = real_imgs.to(device)
        b_size = real_imgs.size(0)

        real_labels = torch.ones(b_size, 1).to(device)
        fake_labels = torch.zeros(b_size, 1).to(device)

        optimizerD.zero_grad()

        output_real = netD(real_imgs)
        loss_real = criterion(output_real, real_labels)

        noise = torch.randn(b_size, noise_dim).to(device)
        fake_imgs = netG(noise)
        output_fake = netD(fake_imgs.detach())
        loss_fake = criterion(output_fake, fake_labels)

        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizerD.step()

        optimizerG.zero_grad()

        output = netD(fake_imgs)
        loss_G = criterion(output, real_labels)
        loss_G.backward()
        optimizerG.step()

    d_acc = (output_real > 0.5).float().mean().item() * 100

    print(
        f"Epoch {epoch}/{epochs} | "
        f"D_loss: {loss_D.item():.4f} | "
        f"D_acc: {d_acc:.2f}% | "
        f"G_loss: {loss_G.item():.4f}"
    )

    print_generator_weights_and_gradients(netG)

    if epoch % save_interval == 0 or epoch == 1:
        save_image(
            fake_imgs[:25],
            f"generated_samples_{dataset_choice}/epoch_{epoch:03d}.png",
            nrow=5,
            normalize=True
        )

## Saving Final Generated Images

In [ ]:
print("\nSaving final 100 generated images...")

netG.eval()
with torch.no_grad():
    z = torch.randn(100, noise_dim).to(device)
    samples = netG(z)

    for i in range(100):
        save_image(
            samples[i],
            f"final_generated_images_{dataset_choice}/img_{i+1}.png",
            normalize=True
        )

## Saving Trained Models

In [ ]:
torch.save(netG.state_dict(), f"generator_{dataset_choice}.pth")
torch.save(netD.state_dict(), f"discriminator_{dataset_choice}.pth")

print("Models saved successfully!")

## Downloading Generated Results


In [ ]:
import os
import zipfile

zip_name = "mnist_gan_outputs"

files_to_zip = [
    "final_generated_images_mnist",
    "generated_samples_mnist",
    "discriminator_mnist.pth",
    "generator_mnist.pth"
]

with zipfile.ZipFile(f"{zip_name}.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for item in files_to_zip:
        if os.path.isdir(item):
            for foldername, subfolders, filenames in os.walk(item):
                for filename in filenames:
                    file_path = os.path.join(foldername, filename)
                    zipf.write(file_path)
        else:
            zipf.write(item)

print(f"{zip_name}.zip created successfully!")

## Conclusion

In this experiment, a Generative Adversarial Network (GAN) was successfully trained on the MNIST and Fashion-MNIST datasets.

The Generator learned to generate realistic images, while the Discriminator learned to differentiate between real and generated images. Model performance was monitored using loss values and layer-wise weight and gradient norms.